<a href="https://colab.research.google.com/github/SergChupin/NLP/blob/lesson_04/nlp_hw_04.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


Урок 4. Тематическое моделирование. EM-алгоритм Задача тематическое моделирование

продолжаем исследование датасета с твитами

Скачиваем датасет (источник): положительные, отрицательные.

рабочие ссылки с твитами https://disk.yandex.ru/i/v5HM-ENiGXZVpQ https://disk.yandex.ru/i/koR5eMCToCZS2Q

как альтернатива можно скачать данные из Роспотребнадзора https://zpp.rospotrebnadzor.ru/Forum/Appeals для этого берём ноутбук parse_rospotrebnadzor.ipynb устанавливаем количество скачанных страниц больше не 50-сят хотябы 500 и для анализа берём только вопросы так как ответы есть не всегда

что надо сделать

объединить в одну выборку (это только для твитов), для роспотребнадзора сформировать датасет из вопросов
провести исследование и выявить тематики о которых говорят в твитах (для твитов), а для роспотребнадзора так же выявить тематики о которых люди пишут проанализировать
сделать визуализацию кластеров тематик
проинтерпритировать получившиеся тематики

In [1]:
! pip install pymorphy2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 55 kB 2.4 MB/s 
     |████████████████████████████████| 8.2 MB 9.6 MB/s 
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13723 sha256=63c933bdf5ec4b5474b216e66455e4596fde6c8ac167d1f9f27ebb2a3efd9720
  Stored in directory: /root/.cache/pip/wheels/72/b0/3f/1d95f96ff986c7dfffe46ce2be4062f38ebd04b506c77c81b9
Successfully built docopt


In [2]:
!pip install -U gensim

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 24.1 MB 1.4 MB/s 
  Attempting uninstall: gensim
    Found existing installation: gensim 3.6.0
    Uninstalling gensim-3.6.0:
      Successfully uninstalled gensim-3.6.0


In [3]:
!pip install pyLDAvis

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.7 MB 5.1 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
  Created wheel for pyLDAvis: filename=pyLDAvis-3.3.1-py2.py3-none-any.whl size=136898 sha256=f801ee4d929b9676bf0c45244bd91782f36b6c242f208443f8b00e824faef180
  Stored in directory: /root/.cache/pip/wheels/c9/21/f6/17bcf2667e8a68532ba2fbf6d5c72fdf4c7f7d9abfa4852d2f
  Created wheel for sklearn: filename=sklearn-0.0-py2.py3-none-any.whl size=1310 sha256=88cc8f10afde05917cd4d12242fc5a130090d06b9c281efc6ffd5fd1a2ca033b
  Stored in directory: /root/.cache/pip/wheels/46/ef/c3/157e41f5ee1372d1be90b09f74f82b10e391eaacca8f22d33e
Successfully built pyLDAvis sklearn


In [4]:
import pandas as pd
import numpy as np

import re
import nltk
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger_ru')
import pymorphy2

from nltk.corpus import stopwords
from tqdm.notebook import tqdm
from multiprocessing import Pool
from pymystem3 import Mystem
from gensim.models import *
from gensim import corpora
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis

from sklearn.metrics import *
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package averaged_perceptron_tagger_ru to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger_ru.zip.
/usr/local/lib/python3.7/dist-packages/past/types/oldstr.py:5: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Iterable
/usr/local/lib/python3.7/dist-packages/past/builtins/misc.py:4: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Mapping


In [5]:
positive = pd.read_csv('/content/positive.csv', sep=';', usecols=[3], names=['text'])
negative = pd.read_csv('/content/negative.csv', sep=';', usecols=[3], names=['text'])
data = positive.append(negative)

In [6]:
data.head()

,text
0,"@first_timee хоть я и школота, но поверь, у на..."
1,"Да, все-таки он немного похож на него. Но мой ..."
2,RT @KatiaCheh: Ну ты идиотка) я испугалась за ...
3,"RT @digger2912: ""Кто то в углу сидит и погибае..."
4,@irina_dyshkant Вот что значит страшилка :D\nН...


In [7]:
morph = pymorphy2.MorphAnalyzer()

/usr/local/lib/python3.7/dist-packages/pymorphy2/units/base.py:70: DeprecationWarning: inspect.getargspec() is deprecated since Python 3.0, use inspect.signature() or inspect.getfullargspec()
  args, varargs, kw, default = inspect.getargspec(cls.__init__)
/usr/local/lib/python3.7/dist-packages/pymorphy2/units/base.py:70: DeprecationWarning: inspect.getargspec() is deprecated since Python 3.0, use inspect.signature() or inspect.getfullargspec()
  args, varargs, kw, default = inspect.getargspec(cls.__init__)
/usr/local/lib/python3.7/dist-packages/pymorphy2/units/base.py:70: DeprecationWarning: inspect.getargspec() is deprecated since Python 3.0, use inspect.signature() or inspect.getfullargspec()
  args, varargs, kw, default = inspect.getargspec(cls.__init__)
/usr/local/lib/python3.7/dist-packages/pymorphy2/units/base.py:70: DeprecationWarning: inspect.getargspec() is deprecated since Python 3.0, use inspect.signature() or inspect.getfullargspec()
  args, varargs, kw, default = inspect.g

In [8]:
words_regex = re.compile('\w+')

def find_words(text, regex = words_regex):
    tokens =  regex.findall(text.lower())
    return [w for w in tokens if w.isalpha() and len(w) >= 3]


stopwords_list = stopwords.words('russian')


add_stop_words = ['ахах', 'мочь', 'супер', 'хотяб', 'ахахаххах', 'аахха', 'настюха',
                  'оля', 'блин']
stopwords_list += add_stop_words



def lemmatize(words, lemmer = morph, stopwords = stopwords_list):
    lemmas = [lemmer.parse(w)[0].normal_form for w in words]
    return [w for w in lemmas if not w in stopwords 
            and w.isalpha()]


def preprocess(text):
    result = (lemmatize(find_words(text)))
    return nltk.pos_tag(result, lang='rus')

<>:1: DeprecationWarning: invalid escape sequence \w
<>:1: DeprecationWarning: invalid escape sequence \w
<>:1: DeprecationWarning: invalid escape sequence \w
<ipython-input-8-a475cf940328>:1: DeprecationWarning: invalid escape sequence \w
  words_regex = re.compile('\w+')


In [9]:
data.text.iloc[10]

'Люблю маму и папу!!!!а в остальное я так...-влюбляюсь, привязываюсь)))\xa0'

In [10]:
print(preprocess(data.text.iloc[10]))

[('любить', 'V'), ('мама', 'S'), ('папа', 'S'), ('остальной', 'V'), ('влюбляться', 'V'), ('привязываться', 'V')]


In [11]:
preprocessed_text = list(tqdm(map(preprocess, data.text), total=len(data)))

  0%|          | 0/226834 [00:00<?, ?it/s]

In [12]:
preprocessed_text[:5]

[[('школотый', 'A=m'),
  ('поверь', 'S'),
  ('самый', 'A-PRO=m'),
  ('общество', 'S'),
  ('профилировать', 'V'),
  ('предмет', 'S'),
  ('тип', 'S')],
 [('всё', 'S-PRO'),
  ('таки', 'PART'),
  ('немного', 'ADV'),
  ('похожий', 'A=m'),
  ('мальчик', 'S'),
  ('всё', 'S-PRO'),
  ('равно', 'A=n'),
  ('хороший', 'A=m')],
 [('katiacheh', 'NONLEX'), ('идиотка', 'S'), ('испугаться', 'V')],
 [('угол', 'S'),
  ('сидеть', 'V'),
  ('погибать', 'V'),
  ('голод', 'S'),
  ('ещё', 'ADV'),
  ('порция', 'S'),
  ('взять', 'V'),
  ('хотя', 'PART'),
  ('жрать', 'V'),
  ('хотеть', 'V'),
  ('http', 'NONLEX')],
 [('значит', 'V'),
  ('страшилка', 'S'),
  ('посмотреть', 'V'),
  ('всё', 'ADV'),
  ('часть', 'S'),
  ('создаться', 'V'),
  ('ощущение', 'S'),
  ('автор', 'S'),
  ('курить', 'V')]]

In [13]:
def only_nouns(preprocessed_text):
    token_list = []
    for tweet in preprocessed_text:
        token_list.append([w[0] for w in tweet if w[1]=='S'])
    return token_list

In [14]:
data['text'] = only_nouns(preprocessed_text=preprocessed_text)
data.sample(5)

,text
4012,[]
41331,[]
60092,[семёрка]
20655,[камрад]
93373,"[ток, минь, беда]"


In [15]:
data = data[data['text'].map(lambda d: len(d)) > 2]

In [16]:
data.head()

,text
0,"[поверь, общество, предмет, тип]"
3,"[угол, голод, порция]"
4,"[страшилка, часть, ощущение, автор]"
14,"[обзор, версия, мусор]"
15,"[аспирантура, наука, добро]"


In [17]:
dictionary = corpora.Dictionary(data['text'])

dictionary.filter_extremes(no_below = 10, no_above = 0.9, keep_n=None) # игнорируем слова, которые встречаются реже 10 раз или составляют более 0.9 словаря 
dictionary.save('tweet.dict')

In [18]:
corpus = [dictionary.doc2bow(text) for text in data['text']]
corpora.MmCorpus.serialize('tweet.model', corpus)

In [19]:
%time

lda = ldamodel.LdaModel(corpus, id2word=dictionary, num_topics=3, chunksize=200, update_every=1, passes=2)

CPU times: user 5 µs, sys: 0 ns, total: 5 µs
Wall time: 8.82 µs


In [20]:
lda.show_topics(num_topics=5, num_words=10, formatted=True)

[(0,
  '0.031*"год" + 0.026*"дом" + 0.012*"экзамен" + 0.011*"снег" + 0.011*"фотка" + 0.010*"сон" + 0.010*"боль" + 0.010*"вечер" + 0.010*"бог" + 0.009*"пиздец"'),
 (1,
  '0.070*"день" + 0.027*"час" + 0.026*"школа" + 0.020*"мама" + 0.018*"ночь" + 0.018*"работа" + 0.016*"урок" + 0.015*"неделя" + 0.015*"голова" + 0.012*"дело"'),
 (2,
  '0.035*"человек" + 0.026*"жизнь" + 0.023*"время" + 0.022*"утро" + 0.016*"бля" + 0.014*"улица" + 0.011*"сука" + 0.010*"учёба" + 0.010*"нога" + 0.009*"конец"')]